In [25]:
#!/usr/bin/python
#
# Created by Albert Au Yeung (2010)
#
# An implementation of matrix factorization
#
try:
    import numpy
except:
    print("This implementation requires the numpy module.")
    exit(0)

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, avg, bi, bu, steps=100, alpha=0.002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - avg - bu[i] - bi[j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T, alpha

def getAvg(R):
    sum = 0
    count = 0
    for i in R :
        for x in i:
            if x > 0:
                sum += x
                count += 1
    return sum/count

def getUserBias(R,avg):
    userBias = []
    for i in R:
        sum = 0
        count = 0
        for x in i:
            if x != 0:
                sum += x
                count += 1
        userBias.append((sum/count) - avg)
    return numpy.array(userBias)
        
def getItemBias(R,avg):
    itemBias = []
    for i in range(len(R[0])):
        sum = 0
        count = 0
        for x in range(len(R)):
            if R[x][i] != 0:
                sum += R[x][i]
                count += 1
        itemBias.append((sum/count)-avg)
    return numpy.array(itemBias)
    
def sumR(R, avg, bu, bi):
    for i in range(len(R)):
        for x in range(len(R[0])):
            if R[i][x] > 0:
                R[i][x] = R[i][x] + avg + bu[i] + bi[x] 
    return R

def getMSE(cek, arr):
    count = 0
    sum = 0
    for i in range(len(arr)):
        for x in range(len(arr[0])):
            if cek[i][x] > 0:
                count += 1
                sum += (arr[i][x] - cek[i][x])**2
    return sum/count             
                
###############################################################################

if __name__ == "__main__":
    cek = [
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 3, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [5, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 5, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
           [3, 0, 5, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
           [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 5, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0],
           [4, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 5, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0]
        ]
    R = [
            [0,0,3,0,0,1,0,0,0,0,4,0,5,0,5,0,0,0,0,0],
            [4,3,0,2,0,2,3,0,0,0,0,0,1,3,0,4,0,0,0,0],
            [0,5,3,1,0,1,0,0,1,0,5,4,0,0,5,0,0,0,0,0],
            [5,0,0,1,3,2,0,4,0,3,4,0,2,3,0,0,0,0,0,0],
            [2,0,3,0,0,3,1,3,1,0,0,0,4,0,0,0,0,0,0,0],
            [0,0,2,0,0,0,0,0,1,0,0,0,0,3,4,4,0,0,0,0],
            [5,0,4,2,0,1,0,5,0,0,0,0,0,0,0,0,0,0,5,0],
            [5,0,4,0,0,1,0,4,0,3,0,0,3,0,3,5,0,0,0,0],
            [0,0,0,0,0,0,4,0,0,0,0,5,0,2,5,0,2,0,0,0],
            [3,0,4,0,0,2,0,0,1,0,2,0,4,0,0,0,0,0,0,1],
            [2,0,5,0,0,0,2,0,0,0,0,3,0,0,5,0,0,0,0,2],
            [4,5,0,0,0,0,0,0,1,0,0,0,3,4,5,4,0,0,0,0],
            [5,0,0,0,4,3,5,0,0,2,0,4,0,5,0,0,0,4,0,2],
            [0,0,3,0,0,0,0,4,5,0,0,0,5,0,5,0,0,0,0,0],
            [4,0,0,2,0,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,3,0,0,0,4,0,2,0,0,0,4,0,5,0,0,3,0,0],
            [1,0,4,3,0,0,2,2,0,1,0,0,5,4,0,0,0,0,0,1],
            [3,0,4,2,4,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [4,0,3,0,0,1,4,5,0,3,5,0,0,0,4,0,0,0,0,0],
            [0,1,0,1,0,2,0,3,0,1,0,5,0,0,5,0,0,0,1,0],
            [0,0,5,2,5,1,3,0,0,0,1,4,0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,4,0,1,2,5,0,0,0,5,0,3,0,2,2],
            [0,0,5,0,4,0,0,0,0,0,3,5,0,5,0,0,0,0,0,0],
            [0,3,0,0,4,0,0,3,0,1,0,0,0,4,0,0,0,1,0,1],
            [3,5,4,1,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,2],
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0,2,3],
            [0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,3,2,2]
        ]

    avg = getAvg(R)
    
    bu = getUserBias(R,avg)
    
    bi = getItemBias(R,avg)

    R = sumR(numpy.array(R), avg, bu, bi)

    N = len(R)
    M = len(R[0])
    K = 2

    P = numpy.random.rand(N,K)
    Q = numpy.random.rand(M,K)

    nP, nQ, alpha = matrix_factorization(R, P, Q, K, avg, bi, bu)
    
    MF = numpy.dot(nP, nQ.T)
    mse = getMSE(cek, MF)
    rmse = mse**0.5
    
    print("the Original Matrix")
    print(R)
    print("the Approximation matrix by MF")
    print(MF)
    print("Parameter = K : ",K, "| alpha : ",alpha)
    print("MSE       = ",mse)
    print("RMSE      = ",rmse)

the Original Matrix
[[ 0  0  7  0  0  3  0  0  0  0  8  0  9  0  9  0  0  0  0  0]
 [ 7  6  0  3  0  3  5  0  0  0  0  0  4  6  0  7  0  0  0  0]
 [ 0  8  6  2  0  2  0  0  2  0  8  8  0  0  9  0  0  0  0  0]
 [ 8  0  0  2  6  3  0  7  0  4  7  0  5  6  0  0  0  0  0  0]
 [ 4  0  6  0  0  4  3  5  1  0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  2  0  0  0  0  6  8  7  0  0  0  0]
 [ 9  0  8  4  0  3  0  9  0  0  0  0  0  0  0  0  0  0  7  0]
 [ 9  0  8  0  0  3  0  8  0  5  0  0  6  0  7  9  0  0  0  0]
 [ 0  0  0  0  0  0  7  0  0  0  0  9  0  6  9  0  4  0  0  0]
 [ 5  0  7  0  0  3  0  0  1  0  4  0  6  0  0  0  0  0  0  2]
 [ 5  0  8  0  0  0  5  0  0  0  0  7  0  0  9  0  0  0  0  3]
 [ 8  9  0  0  0  0  0  0  3  0  0  0  7  8  9  8  0  0  0  0]
 [ 9  0  0  0  8  5  8  0  0  4  0  8  0  9  0  0  0  7  0  4]
 [ 0  0  7  0  0  0  0  8  7  0  0  0  9  0 10  0  0  0  0  0]
 [ 8  0  0  4  0  5  8  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 9  0  7  0  0  0  7  0  4  0  0 